In [1]:
import matplotlib.pyplot as plt

import torchvision as tv
import torch.nn.functional as F
import torch
import torch.nn as nn
from torch.autograd import Variable
import torchvision.models as models
from torchvision import transforms, utils
from torch.utils.data import Dataset, DataLoader
from PIL import Image
import numpy as np
import torch.optim as optim
import os
import time
import loadData8
import itertools
from sklearn.metrics import confusion_matrix
import numpy as np

In [2]:
device = torch.device("cuda:0")

In [3]:
transforms = tv.transforms.Compose([
    # transforms.CenterCrop(64),
    # transforms.CenterCrop(128),
    transforms.ToTensor(),
    # transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5)),
])

In [4]:
class Net8(nn.Module):
    def __init__(self):
        super(Net8, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (1, 10))
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = (2, 3))
        
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(20304, 1024)
        self.fc2 = nn.Linear(1024, 500)
        self.fc3 = nn.Linear(500, 54)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
    
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
    
net8 = Net8()
path = './model_10dB_8个0.pkl'
net8.load_state_dict(torch.load(path))
net8 = net8.to(device)

In [5]:
class Net0(nn.Module):
    def __init__(self):
        super(Net0, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (1, 10))
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = (2, 3))
        
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(2384, 1024)
        self.fc2 = nn.Linear(1024, 500)
        self.fc3 = nn.Linear(500, 54)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
    
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output
    
net0 = Net0()
path = './model_10dB_1个0.pkl'
net0.load_state_dict(torch.load(path))
net0 = net0.to(device)

In [6]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.conv1 = nn.Conv2d(in_channels = 1, out_channels = 8, kernel_size = (1, 10))
        self.conv2 = nn.Conv2d(in_channels = 8, out_channels = 16, kernel_size = (2, 3))
        
        self.dropout1 = nn.Dropout(0.5)
        self.dropout2 = nn.Dropout(0.5)
        
        self.fc1 = nn.Linear(10064, 1024)
        self.fc2 = nn.Linear(1024, 500)
        self.fc3 = nn.Linear(500, 54)

    def forward(self, x):
        x = self.conv1(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, 1)
    
        x = torch.flatten(x, 1)
        x = self.fc1(x)
        x = F.relu(x)
        x = self.dropout1(x)
        x = self.fc2(x)
        x = F.relu(x)
        x = self.dropout2(x)
        x = self.fc3(x)
        output = F.log_softmax(x, dim=1)
        return output

net = Net()
path = './model_10dB_4个0.pkl'
net.load_state_dict(torch.load(path))
net = net.to(device)

In [7]:
predict1, predict2, predict3 = [],[],[]
originLabels = []

In [8]:
import deleteFolder
for line in open("files_test10.txt"):   
    print(line)
    lines = line.split()
    locate, label = lines[0], lines[1]
    originData = np.load(locate)
    
    #测试8个0的情况
    with open("./split-no.txt" ,"w") as f8:
        f8.write(line)
    
    batchSize = 4
    test_data = loadData8.MyDataset(txt=f'split-no.txt', transform=None)
    test_loader = DataLoader(dataset=test_data, batch_size=batchSize, shuffle=True, num_workers=2)

    for data in test_loader:
        inputs, labels = data
        originLabel = int(labels)
        originLabels.append(originLabel)

        a = inputs[0:160]
#         print("---", a.shape)
        inputs, labels = inputs.cuda(), labels.cuda()
        inputs = torch.tensor(inputs, dtype=torch.float32, device = device)
        outputs = net8(inputs)
        _, predicted = torch.max(outputs.data, dim=1)
        predict1.append(int(predicted[0]))
#         print(labels,"---", predicted)
    
    
    #测试输入1个0的情况
    splitData = np.split(originData,8,axis=1)
    for i in range(len(splitData)):
#         print(splitData[i].shape)
        path1 = "/workspace/模型混合/输入1个0的中间数据/" + str(i) + ".npy"
        np.save(path1, splitData[i])
        
    with open("./split8.txt" ,"w") as f_split8:
        for i in range(len(splitData)):
            path1 = "/workspace/模型混合/输入1个0的中间数据/" + str(i) + ".npy"
            f_split8.write(f'{path1} {originLabel}\n')
            
    batchSize = 4
    test_data = loadData8.MyDataset(txt=f'split8.txt', transform=None)
    test_loader = DataLoader(dataset=test_data, batch_size=batchSize, shuffle=True, num_workers=2)
    
    
    predictedLabels = []
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        inputs = torch.tensor(inputs, dtype=torch.float32, device = device)
        outputs = net0(inputs)
        _, predicted = torch.max(outputs.data, dim=1)
        predictedLabels.append([int(label) for label in predicted])
    predictedLabels = np.reshape(predictedLabels, (1, -1))[0]
#     print("predictedLabels:", predictedLabels)
#     predict2.append(np.argmax(np.bincount(predictedLabels)))
    predict2.append([predictedLabel for predictedLabel in predictedLabels])
        
    #测试输入4个0的情况
    originData = np.load(locate)
    splitData = np.split(originData,2,axis=1)
    for i in range(len(splitData)):
#         print(splitData[i].shape)
        path1 = "/workspace/模型混合/输入4个0的中间数据/" + str(i) + ".npy"
        np.save(path1, splitData[i])
    
    with open("./split2.txt" ,"w") as f_split8:
        for i in range(len(splitData)):
            path1 = "/workspace/模型混合/输入4个0的中间数据/" + str(i) + ".npy"
            f_split8.write(f'{path1} {originLabel}\n')
    
    batchSize = 4
    test_data = loadData8.MyDataset(txt=f'split2.txt', transform=None)
    test_loader = DataLoader(dataset=test_data, batch_size=batchSize, shuffle=True, num_workers=2)
    
    
    predictedLabels = []
    for data in test_loader:
        inputs, labels = data
        inputs, labels = inputs.cuda(), labels.cuda()
        inputs = torch.tensor(inputs, dtype=torch.float32, device = device)
        outputs = net(inputs)
        _, predicted = torch.max(outputs.data, dim=1)
        predictedLabels.append([int(label) for label in predicted])
    predictedLabels = np.reshape(predictedLabels, (1, -1))[0]
#     print("predictedLabels:", predictedLabels)
    predict3.append([p for p in predictedLabels]) 
   
    
    print(originLabels[-1],  predict2[-1], predict3[-1], predict1[-1])

/workspace/时域-8个0一组的实验/10dB处理后的数据/10dBProcess_A_10_120.npy 9



/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:24: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:52: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).
/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:83: UserWarning: To copy construct from a tensor, it is recommended to use sourceTensor.clone().detach() or sourceTensor.clone().detach().requires_grad_(True), rather than torch.tensor(sourceTensor).


9 [25, 9, 9, 39, 27, 5, 33, 28] [32, 9] 9
/workspace/时域-8个0一组的实验/10dB处理后的数据/10dBProcess_A_10_121.npy 9

9 [25, 7, 9, 38, 11, 7, 10, 28] [9, 9] 7
/workspace/时域-8个0一组的实验/10dB处理后的数据/10dBProcess_A_10_122.npy 9

9 [39, 10, 6, 9, 9, 5, 26, 7] [9, 9] 9
/workspace/时域-8个0一组的实验/10dB处理后的数据/10dBProcess_A_10_123.npy 9

9 [7, 7, 5, 36, 7, 9, 1, 5] [9, 9] 9
/workspace/时域-8个0一组的实验/10dB处理后的数据/10dBProcess_A_10_124.npy 9

9 [8, 11, 43, 10, 28, 33, 7, 6] [9, 33] 9
/workspace/时域-8个0一组的实验/10dB处理后的数据/10dBProcess_A_10_125.npy 9

9 [8, 9, 38, 25, 9, 10, 9, 38] [9, 9] 9
/workspace/时域-8个0一组的实验/10dB处理后的数据/10dBProcess_A_10_126.npy 9

9 [28, 11, 7, 33, 38, 0, 25, 10] [9, 9] 9
/workspace/时域-8个0一组的实验/10dB处理后的数据/10dBProcess_A_10_127.npy 9

9 [39, 38, 9, 28, 27, 11, 33, 9] [9, 9] 9
/workspace/时域-8个0一组的实验/10dB处理后的数据/10dBProcess_A_10_128.npy 9

9 [1, 1, 27, 27, 1, 25, 25, 1] [9, 9] 9
/workspace/时域-8个0一组的实验/10dB处理后的数据/10dBProcess_A_10_129.npy 9

9 [38, 34, 38, 10, 8, 33, 9, 6] [9, 9] 8
/workspace/时域-8个0一组的实验/10dB处理后的数据/10

In [9]:
originLabels[0]

9

In [10]:
predict1[0]

9

In [11]:
predict2[0]

[25, 9, 9, 39, 27, 5, 33, 28]

In [12]:
predict3[0]

[32, 9]

In [13]:
print(len(originLabels), len(predict1), len(predict2), len(predict3))

1674 1674 1674 1674


In [14]:
rightNum = 0
for i in range(len(originLabels)):
    predictAll = []
    
    for _ in range(2): # 1个0一组的权重
        for a in predict2[i]:
            predictAll.append(a)
    
    for _ in range(3): # 4个0一组的权重
        for a in predict3[i]:
            predictAll.append(a)
            
    for _ in range(4): # 8个0一组的权重
        predictAll.append(predict1[i])
#     print("-----",predictAll)
#     print(p.shape)
    result = np.argmax(np.bincount(predictAll))
#     print(result, originLabels[i])
    if originLabels[i] == result:
        rightNum += 1
    else:
        print(originLabels[i], predict1[i], predict2[i], predict3[i])
print(rightNum/ len(originLabels))

9 7 [25, 7, 9, 38, 11, 7, 10, 28] [9, 9]
9 10 [9, 8, 38, 10, 39, 10, 10, 48] [10, 9]
9 10 [10, 9, 25, 6, 38, 9, 10, 36] [9, 10]
10 9 [9, 25, 28, 8, 10, 8, 7, 7] [8, 10]
10 28 [28, 3, 28, 10, 41, 28, 10, 8] [10, 28]
10 10 [31, 28, 9, 1, 7, 28, 31, 7] [9, 38]
10 10 [3, 32, 38, 10, 3, 25, 3, 9] [9, 8]
11 39 [39, 39, 39, 11, 3, 11, 27, 39] [39, 11]
11 39 [27, 4, 11, 39, 11, 36, 11, 39] [11, 39]
12 23 [53, 15, 20, 15, 23, 16, 15, 12] [15, 20]
12 12 [44, 14, 50, 53, 12, 20, 14, 18] [23, 14]
12 12 [12, 12, 44, 14, 12, 44, 15, 44] [44, 44]
12 12 [12, 14, 14, 14, 16, 16, 24, 12] [14, 44]
12 12 [24, 53, 44, 49, 24, 12, 24, 24] [24, 44]
12 14 [15, 15, 20, 12, 24, 23, 23, 44] [15, 12]
13 40 [42, 18, 49, 50, 18, 18, 23, 50] [49, 15]
13 19 [22, 42, 47, 13, 23, 19, 21, 13] [51, 18]
13 14 [20, 24, 50, 18, 20, 41, 19, 22] [19, 23]
13 19 [40, 19, 50, 13, 13, 23, 19, 13] [13, 19]
13 12 [50, 49, 13, 49, 18, 15, 20, 23] [15, 20]
14 20 [13, 51, 14, 40, 23, 22, 50, 12] [16, 41]
14 14 [16, 16, 19, 19, 18, 41,

In [15]:
# 加权值
